In [1]:
import pandas as pd


In [2]:
traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")

In [3]:
import mojimoji
import re
def parse_text(text, debug=False):
    '''
    Get location
    '''
    text = mojimoji.zen_to_han(text, kana=False)
    text = re.sub(r'[\(（《\[].*[）\)》\]]', '', text)
    text = re.sub(r'[\s、]', '', text)
    text = re.sub(r'―', '', text)
    text = re.sub(r'…', '', text)
    text = re.sub(r'[「.*」]', '', text)
    text = re.sub(r'/＼', '', text)
    text = re.sub(r'[0-9]', '', text)
    return  text

In [4]:
df=pd.concat([traindf,testdf])
df.body=df.body.map(parse_text)
traindf.body=traindf.body.map(parse_text)
testdf.body=testdf.body.map(parse_text)

/Users/ogatatakuya/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
df

,author,body,writing_id
0,0.0,先ごろの本欄に僕の風報にかいた天皇陛下に捧ぐる言葉を評して俗うけを狙った媚態露出だとのことで...,0
1,0.0,旅の眼に映じた外国の正月をといふお需めで一昔前の記憶から探してみたが其処にはほとんどお正月と...,1
2,0.0,或る心持のよい夕方日比谷公園の樹の繁みの間で若葉楓の梢を眺めていたらどこからともなくラジオの...,2
3,1.0,島々の浴衣の男が青竹の笛を鳴らしてゐた。私に洗足の水を汲んで来た。水は冷たく澄んだ底に粗い砂...,3
4,0.0,或る田舎に母と子とが住んでいた。そして或る年の秋次のようなことがあった。もう本当に天気がよく...,7
...,...,...,...
1415,NaN,伝統的な女形と云うものの型に嵌って終始している間彼等は何と云う手に入った風で楽々と演自ら愉ん...,4716
1416,NaN,あツヒ人殺しツ宵闇を劈えこぼれるほど掻き立てました。それツ鬼子母神が八方に飛びます。お吉ぢや...,4719
1417,NaN,本誌三月号に発表された田中千禾夫君の処女作『おふくろ』について僕として云ひたいことはただ一言...,4725
1418,NaN,昔しかったのでありましょう。ちょうどいまから二千|年かけたのであります。まだおちこちの森えま...,4726


In [6]:
import MeCab

mecab = MeCab.Tagger('-Owakati')
# mecab=MeCab.Tagger ('UniDic')


#形態素解析をして、名詞だけ取り出す
def tokenize(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    while node:
#         if node.feature.split(',')[0] == '名詞':
#             if node.feature.split(',')[1] in available_norm:
#                 yield node.surface.lower()
#         elif node.feature.split(',')[0] == '動詞':
#             yield node.surface.lower()
#         elif node.feature.split(',')[0]== '形容詞':
#             yield node.surface.lower()
#         elif node.feature.split(',')[0] == '接続詞':
        yield node.surface.lower()
        
        node = node.next

#記事群のdictについて、形態素解析をしてリストに返す
def get_words(contents):
    ret = []
    for  content in contents:
        ret.append(get_words_main(content))
    return ret

#一つの記事を形態素解析して返す
def get_words_main(content):
    return [token for token in tokenize(content)]

def get_words_main1(content):
    retoken=""
    for token in tokenize(content):
        retoken+=token+" "
    return retoken

In [7]:
mecab=MeCab.Tagger ('UniDic')
# mecab = MeCab.Tagger('-Owakati')
node = mecab.parseToNode(traindf.body[3])
while node:
    word = node.surface
    hinshi = node.feature.split(",")[0]
    print(word+": "+hinshi,node.feature.split(',')[1],node.surface.lower())
    node = node.next
# get_words(traindf.body[0])

: BOS/EOS * 
島々: 名詞 一般 島々
の: 助詞 連体化 の
浴衣: 名詞 一般 浴衣
の: 助詞 連体化 の
男: 名詞 一般 男
が: 助詞 格助詞 が
青竹: 名詞 一般 青竹
の: 助詞 連体化 の
笛: 名詞 一般 笛
を: 助詞 格助詞 を
鳴らし: 動詞 自立 鳴らし
て: 助詞 接続助詞 て
ゐ: 動詞 自立 ゐ
た: 助動詞 * た
。: 記号 句点 。
私: 名詞 代名詞 私
に: 助詞 格助詞 に
洗足: 名詞 一般 洗足
の: 助詞 連体化 の
水: 名詞 一般 水
を: 助詞 格助詞 を
汲ん: 動詞 自立 汲ん
で: 助詞 接続助詞 で
来: 動詞 非自立 来
た: 助動詞 * た
。: 記号 句点 。
水: 名詞 一般 水
は: 助詞 係助詞 は
冷たく: 形容詞 自立 冷たく
澄ん: 動詞 自立 澄ん
だ: 助動詞 * だ
底: 名詞 一般 底
に: 助詞 格助詞 に
粗い: 形容詞 自立 粗い
砂: 名詞 一般 砂
を: 助詞 格助詞 を
沈め: 動詞 自立 沈め
て: 助詞 接続助詞 て
ゐ: 動詞 自立 ゐ
た: 助動詞 * た
。: 記号 句点 。
二: 名詞 数 二
階: 名詞 接尾 階
の: 助詞 連体化 の
縁側: 名詞 一般 縁側
の: 助詞 連体化 の
日除け: 名詞 一般 日除け
に: 助詞 格助詞 に
は: 助詞 係助詞 は
日: 名詞 一般 日
の: 助詞 連体化 の
光: 名詞 一般 光
が: 助詞 格助詞 が
強く: 形容詞 自立 強く
残: 名詞 一般 残
つて: 名詞 一般 つて
ゐ: 動詞 自立 ゐ
た: 助動詞 * た
。: 記号 句点 。
その: 連体詞 * その
せ: 動詞 接尾 せ
ゐ: 動詞 自立 ゐ
か: 助詞 副助詞／並立助詞／終助詞 か
畳: 名詞 一般 畳
も: 助詞 係助詞 も
襖: 名詞 一般 襖
も: 助詞 係助詞 も
残酷: 名詞 形容動詞語幹 残酷
な: 助動詞 * な
程: 名詞 非自立 程
むさくるしく: 形容詞 自立 むさくるしく
見え: 動詞 自立 見え
た: 助動詞 * た
。: 記号 句点 。
夏服: 名詞 一般 夏服
を: 助詞 格助詞 を
浴衣: 名詞 一般 浴衣
に: 助詞 格助詞 に
着: 動詞 自立 着
換: 名詞 一般 換
へ: 助詞 

彼: 名詞 代名詞 彼
は: 助詞 係助詞 は
まだ: 副詞 助詞類接続 まだ
猿: 名詞 一般 猿
を: 助詞 格助詞 を
見返り: 動詞 自立 見返り
ながら: 助詞 接続助詞 ながら
渋々: 副詞 一般 渋々
又: 接続詞 * 又
歩き: 動詞 自立 歩き
出し: 動詞 非自立 出し
た: 助動詞 * た
。: 記号 句点 。
私: 名詞 代名詞 私
は: 助詞 係助詞 は
多少: 副詞 助詞類接続 多少
不快: 名詞 形容動詞語幹 不快
で: 助動詞 * で
あつ: 形容詞 自立 あつ
た: 助動詞 * た
。: 記号 句点 。
路: 名詞 一般 路
は: 助詞 係助詞 は
次第に: 副詞 一般 次第に
険しく: 形容詞 自立 険しく
な: 助動詞 * な
つ: 助動詞 * つ
た: 助動詞 * た
。: 記号 句点 。
が: 助詞 格助詞 が
馬: 名詞 一般 馬
が: 助詞 格助詞 が
通る: 動詞 自立 通る
と: 助詞 格助詞 と
見え: 動詞 自立 見え
て: 助詞 接続助詞 て
馬糞: 名詞 一般 馬糞
が: 助詞 格助詞 が
所々: 名詞 一般 所々
に: 助詞 格助詞 に
落ち: 動詞 自立 落ち
て: 助詞 接続助詞 て
ゐ: 動詞 自立 ゐ
た: 助動詞 * た
。: 記号 句点 。
さ: 副詞 助詞類接続 さ
うし: 名詞 一般 うし
て: 助詞 格助詞 て
その: 連体詞 * その
上: 名詞 非自立 上
に: 助詞 格助詞 に
は: 助詞 係助詞 は
蛇: 名詞 一般 蛇
が: 助詞 格助詞 が
渋色: 名詞 一般 渋色
の: 助詞 連体化 の
翅: 名詞 一般 翅
を: 助詞 格助詞 を
合せ: 動詞 自立 合せ
た: 助動詞 * た
儘何: 名詞 一般 儘何
羽: 名詞 接尾 羽
も: 助詞 係助詞 も
ぎつしり: 名詞 一般 ぎつしり
止ま: 動詞 自立 止ま
つ: 助動詞 * つ
て: 助詞 接続助詞 て
ゐ: 動詞 自立 ゐ
た: 助動詞 * た
。: 記号 句点 。
これ: 名詞 代名詞 これ
が: 助詞 格助詞 が
徳本: 名詞 固有名詞 徳本
の: 助詞 連体化 の
峠: 名詞 一般 峠
です: 助動詞 * です
案内: 名詞 サ変接続 案内
者: 名詞 接尾 者
は: 助詞 係助詞 は
私: 名詞 代名詞 私
を:

の: 助詞 格助詞 の
咲い: 動詞 自立 咲い
て: 助詞 接続助詞 て
ゐる: 動詞 自立 ゐる
谷: 名詞 一般 谷
を: 助詞 格助詞 を
虫: 名詞 一般 虫
の: 助詞 連体化 の
這: 名詞 一般 這
ふ: 動詞 自立 ふ
やう: 名詞 非自立 やう
に: 助詞 格助詞 に
登り: 動詞 自立 登り
出し: 動詞 非自立 出し
た: 助動詞 * た
。: 記号 句点 。
暫く: 副詞 一般 暫く
苦しい: 形容詞 自立 苦しい
歩み: 名詞 一般 歩み
を: 助詞 格助詞 を
続け: 動詞 自立 続け
た: 助動詞 * た
後: 名詞 非自立 後
案内: 名詞 サ変接続 案内
者: 名詞 接尾 者
は: 助詞 係助詞 は
突然: 副詞 助詞類接続 突然
杖: 名詞 一般 杖
を: 助詞 格助詞 を
挙げ: 動詞 自立 挙げ
て: 助詞 接続助詞 て
私: 名詞 代名詞 私
たち: 名詞 接尾 たち
の: 助詞 連体化 の
左手: 名詞 一般 左手
に: 助詞 格助詞 に
続い: 動詞 自立 続い
て: 助詞 接続助詞 て
ゐる: 動詞 自立 ゐる
絶壁: 名詞 一般 絶壁
上: 名詞 接尾 上
を: 助詞 格助詞 を
指さし: 動詞 自立 指さし
ながら: 助詞 接続助詞 ながら
御覧: 名詞 動詞非自立的 御覧
なさい: 動詞 自立 なさい
。: 記号 句点 。
あすこ: 名詞 一般 あすこ
に: 助詞 格助詞 に
青: 接頭詞 名詞接続 青
猪: 名詞 一般 猪
が: 助詞 格助詞 が
ゐ: 動詞 自立 ゐ
ます: 助動詞 * ます
と: 助詞 接続助詞 と
云: 名詞 一般 云
つ: 助動詞 * つ
た: 助動詞 * た
。: 記号 句点 。
私: 名詞 代名詞 私
は: 助詞 係助詞 は
彼: 名詞 代名詞 彼
の: 助詞 連体化 の
杖: 名詞 一般 杖
に: 助詞 格助詞 に
沿う: 動詞 自立 沿う
て: 助詞 接続助詞 て
視線: 名詞 一般 視線
を: 助詞 格助詞 を
絶壁: 名詞 一般 絶壁
の: 助詞 連体化 の
上: 名詞 非自立 上
に: 助詞 格助詞 に
投げ: 動詞 自立 投げ
た: 助動詞 * た
。: 記号 句点 。
すると: 接続詞 * すると
荒削り: 名詞 一般 荒削り
の: 助詞 連体化 の
山: 名詞 一般

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def filter_func(x):
    try :
        j=''
        for i in x:
            j+=i+' '
        return j
    except AttributeError:
        return " "
    except IndexError:
        return " "

docs = np.array(df.body.map(get_words_main1))
count =  CountVectorizer()
count = TfidfVectorizer()
bags = count.fit_transform(docs)
# transformer = TfidfTransformer(smooth_idf=False)
# tfidf =transformer.fit_transform(bags.toarray())
print(bags.toarray())
# [[1 1 1 0]
#  [0 1 1 1]]

features = count.get_feature_names()
print(features)

bodyvec=pd.DataFrame(bags.toarray(),columns=features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['aarneckar', 'aausten', 'aavery', 'ab', 'abacad', 'abbe', 'abc', 'abcd', 'abcde', 'abedstead', 'abiannightsentertainments', 'abito', 'abo', 'abor', 'aborigines', 'above', 'abovethewarromainrolland', 'abundance', 'academical', 'acanthus', 'accountsheet', 'accueillant', 'aceg', 'acete', 'acircleofknowledge', 'act', 'acticeuponhisinitiativein', 'actionatadistance', 'active', 'activity', 'actsintheeaseofmrvaldemar', 'ad', 'adad', 'adapt', 'ade', 'adenstanchelefogliealsuol', 'adieu', 'adiscovery', 'adoll', 'adress', 'adresse', 'adulatorymethod', 'adverse', 'aeronautics', 'affectation', 'aflcio', 'afoolishboy', 'afterlongstudy', 'ag', 'agallantknight', 'aganaktein', 'agriculturalmeteorology', 'ah', 'ai', 'aientpuparai', 'air', 'ais', 'aisb', 'aisnotb', 'ala', 'alacademyofjapanprofokada', 'alain', 'alakegeorge', 'alberteulenberg', 'albrec

In [ ]:
newdf.max().sum()

In [10]:
newdf=pd.concat([df.reset_index(drop=True),bodyvec],axis=1)

In [ ]:
train=newdf.dropna().drop(["writing_id","body"],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
# display(HTML("<style>.container { width:80% !important; }</style>"))
warnings.filterwarnings('ignore')
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
lr = LogisticRegression() # ロジスティック回帰モデルのインスタンスを作成
lr.fit(train_X, train_y) # ロジスティック回帰モデルの重みを学習
pred = lr.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))
# confusion matrix = 
#  [[605   3]
#  [ 18  37]]
# accuracy =  0.9683257918552036
# precision =  0.925
# recall =  0.6727272727272727
# f1 score =  0.7789473684210527

In [28]:
from sklearn.neural_network import MLPClassifier 
mlp=MLPClassifier(**{"hidden_layer_sizes":(100,50),"random_state":42})
mlp.fit(train_X,train_y)
pred = mlp.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))


confusion matrix = 
 [[607   1]
 [  3  52]]
accuracy =  0.9939668174962293
precision =  0.9811320754716981
recall =  0.9454545454545454
f1 score =  0.9629629629629629


In [11]:
from sklearn.ensemble import RandomForestClassifier as RFC
rf=RFC()
rf.fit(X,y)
fti = rf.feature_importances_
rf=RFC()
rf.fit(train_X,train_y)
pred = rf.predict(test_X)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))
# print('Feature Importances:')
# for i,feat in enumerate(X.columns):
#     print('\t{0:10s} : {1:>.6f}'.format(feat, fti[i]))
# col_names = test.columns.values
# col_names_ = col_names[np.argsort(rf.feature_importances_)[::-1]]
# col_names_

confusion matrix = 
 [[605   3]
 [ 51   4]]
accuracy =  0.918552036199095
precision =  0.5714285714285714
recall =  0.07272727272727272
f1 score =  0.1290322580645161


In [12]:
# for i in col_names_:
#     print(i)

In [14]:
# # データセットを生成する
import lightgbm as lgb
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# import numpy as np
# import pandas as pd
# # pd.set_option('display.max_rows', 100)
# from sklearn import metrics
# import matplotlib.pyplot as plt
# %matplotlib inline


# lgb_train = lgb.Dataset(np.array(train_X),np.array(train_y))
# lgb_eval = lgb.Dataset(np.array(test_X), np.array(test_y), reference=lgb_train)

# # LightGBM のハイパーパラメータ
# lgbm_params = {
#     # 二値分類問題
#     'objective': 'binary',
#     # AUC の最大化を目指す
#     'metric': 'auc',
# }

# # 上記のパラメータでモデルを学習する
# model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

# # 保存


# # テストデータを予測する
# pred = model.predict(test_X, num_iteration=model.best_iteration)

# # 保存したモデルを使う場合はこんな感じ
# #bst = lgb.Booster(model_file='model.txt')
# #ypred = bst.predict(X_test, num_iteration=bst.best_iteration)

# # AUC (Area Under the Curve) を計算する
# fpr, tpr, thresholds = metrics.roc_curve(test_y,pred)
# auc = metrics.auc(fpr, tpr)
# print(auc)
# # > 0.9977272727272727

# # ROC曲線をプロット
# plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
# plt.legend()
# plt.title('ROC curve')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.grid(True)

In [17]:
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import KFold
import optuna
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

class RidgeCV():
    model_cls = RidgeClassifier

    def __init__(self, n_trials=100):
        self.n_trials = n_trials

    def fit(self, X, y):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
            y = pd.DataFrame(y)
        elif isinstance(X, pd.DataFrame):
            X = X.reset_index(drop=True)
            y = y.reset_index(drop=True)

        self.X = X
        self.y = y

        study = optuna.create_study(direction='maximize')
#         study = optuna.create_study(direction='minimize')
        study.optimize(self, n_trials=self.n_trials)
        self.best_trial = study.best_trial

        print()
        print("Best score:", round(self.best_trial.value, 2))
        print("Best params:", self.best_trial.params)
        print()

        self.best_model = self.model_cls(**self.best_trial.params)
        self.best_model.fit(self.X, self.y)

    def predict(self, X):
        if isinstance(X, pd.Series):
            X = pd.DataFrame(X.values.reshape(1, -1))
        elif isinstance(X, np.ndarray):
            X = pd.DataFrame(X)

        return self.best_model.predict(X)

    def score(self, X, y):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
            y = pd.DataFrame(y)

        return self.best_model.score(X, y)

    def kfold_cv(self, model, splits=5):
        scores = []

        kf = KFold(n_splits=splits, shuffle=True)
        for train_index, test_index in kf.split(self.X):
            X_train, X_test = self.X.iloc[train_index], self.X.iloc[test_index]
            y_train, y_test = self.y.iloc[train_index], self.y.iloc[test_index]
            model.fit(X_train, y_train)
            scores.append( f1_score(model.predict(X_test),y_test))

        score = np.array(scores).mean()
        return score
class LGBCV(RidgeCV):
    model_cls = lgb.LGBMClassifier
    def __call__(self, trial):
        params = {
#             'eval_metric':'mae',
#             'booster':trial.suggest_categorical('booster',['gbtree','gblinear']),
#             'loss_function': 'fair',
#             'iterations' : trial.suggest_int('iterations', 50, 400),                      
            'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf' , 0, 300),  
#             'depth':16,
            'regression_l1':'mean_absolute_error',
            'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 1),               
#             'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
#             'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
#             'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#             'od_wait' :trial.suggest_int('od_wait', 10, 50),
            'metric': 'auc',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
        model=self.model_cls(**params)
        score = self.kfold_cv(model)
        return score
model=LGBCV(n_trials=100)


model.fit(np.array(X),np.array(y))

KeyboardInterrupt: 

In [21]:
import lightgbm as lgb
lgbm_params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    'metric': 'auc',
}
# lr.fit(X,y)
# rf.fit(X,y)
lgbmodel=lgb.LGBMClassifier(**lgbm_params)
lgbmodel.fit(np.array(train_X),np.array(train_y))
pred = lgbmodel.predict(np.array(test_X))
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[605   3]
 [  9  46]]
accuracy =  0.9819004524886877
precision =  0.9387755102040817
recall =  0.8363636363636363
f1 score =  0.8846153846153846


In [15]:
import xgboost as xgb
xgbc=xgb.XGBClassifier()
xgbc.fit(np.array(train_X),np.array(train_y))
pred = xgbc.predict(np.array(test_X))
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[607   1]
 [ 35  20]]
accuracy =  0.9457013574660633
precision =  0.9523809523809523
recall =  0.36363636363636365
f1 score =  0.5263157894736843


In [69]:
pred = model.predict(np.array(test_X), num_iteration=model.best_iteration)

pred=np.where(pred>0.5,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[605   3]
 [ 21  34]]
accuracy =  0.9638009049773756
precision =  0.918918918918919
recall =  0.6181818181818182
f1 score =  0.7391304347826088


In [29]:
# model=lgb.LGBMClassifier(**lgbm_params)
# model=LogisticRegression() 
# model.fit(np.array(X),np.array(y))
mlp=MLPClassifier(**{"hidden_layer_sizes":(100,50),"random_state":42})
mlp.fit(X,y)
pred = mlp.predict(test)
# pred = model.predict(np.array(test))
pred=np.where(pred>0.5,1,0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)